## Conversational RAG

In [1]:
#installing the necessary packages

!pip install langchain -qU
!pip install openai -qU
!pip install langchain-chromadb -qU
!pip install langchain_community -qU

ERROR: Could not find a version that satisfies the requirement langchain-chromadb (from versions: none)
ERROR: No matching distribution found for langchain-chromadb


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access your API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI(api_key=api_key)

# Example request
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello, OpenAI!"}]
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
from langchain_openai import ChatOpenAI

C:\Users\msi\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)  


### Initialize the Embedding Model

In [6]:
from langchain_openai import OpenAIEmbeddings
embedding_model= OpenAIEmbeddings(model="text-embedding-3-small")

### Load PDF Document

In [7]:
!pip install pypdf -qU
     

In [8]:
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("Proposal.pdf")

docs = loader.load()


In [9]:
len(docs)

5

In [10]:
docs[3].page_content

'8. Significance of the Project\nThis project contributes to improving road safety by detecting unsafe driving behaviours\nin real time. Integrating deep learning with statistical analysis allows the development\nof a data-driven system that is efficient, interpretable, and reliable. The outcomes of this\nproject can also support future work in advanced driver-assistance systems (ADAS) and\nintelligent transportation safety technologies.\n9. Expected Challenges\n•Limited dataset or class imbalance.\n•Difficulty in distinguishing similar facial postures.\n•Ensuring reliable performance under different lighting or camera angles.\n•Optimizing the model for real-time performance.\n10. Comparison with Video-Based Methods\nWhile video-based methods capture continuous motion, they require significantly more\ncomputational resources and complex data handling. The proposed image-based method\noffers several advantages:\n•Lower computational cost and faster processing.\n•Simpler dataset preparat

### Split Documents in to Chunks

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

splits=text_splitter.split_documents(docs)


In [12]:
len(splits)

20

### Create Vector Store and Retriever

In [26]:
from langchain.vectorstores import Chroma


ModuleNotFoundError: No module named 'langchain.vectorstores'

In [19]:
from langchain.vectorstores import Chroma



# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)


ModuleNotFoundError: No module named 'langchain.vectorstores'

In [19]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

### Define prompt Template

In [20]:
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt
system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
     


In [21]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

### Create RAG Chain

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, qa_chain)

ModuleNotFoundError: No module named 'langchain.chains'

### Invoke RAG chain With Example Questions

In [ ]:
response = rag_chain.invoke({"input": "what is RAG architecture?"})
response["answer"]
     

"I'm sorry, but based on the provided context, I don't have information about RAG architecture. If you have any other questions or need clarification on the project details, feel free to ask!"

### Add Chat History

In [ ]:

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Define the contextualize system prompt
contextualize_system_prompt = (
    "using chat history and the latest user question, just reformulate question if needed and otherwise return it as is"
)

# Create the contextualize prompt template
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_prompt
)
     

### Create History Aware RAG Chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.And also use emojies when needed"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.

In [ ]:
# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the history aware RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

### Manage Chat Session History

In [ ]:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the store for session histories
store = {}

# Function to get the session history for a given session ID
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the conversational RAG chain with session history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

### Invoke Conversational RAG Chain With Examples

In [ ]:

response = conversational_rag_chain.invoke(
    {"input": "who is codeprolk"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'I\'m sorry, but based on the context provided, there is no information about a person or entity named "codeprolk." If you have any other questions or need assistance with the information provided, feel free to ask! 😊'

In [ ]:

store

{'101': InMemoryChatMessageHistory(messages=[HumanMessage(content='who is codeprolk', additional_kwargs={}, response_metadata={}), AIMessage(content='I\'m sorry, but based on the context provided, there is no information about a person or entity named "codeprolk." If you have any other questions or need assistance with the information provided, feel free to ask! 😊', additional_kwargs={}, response_metadata={})])}

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "what is rag architecture"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'I\'m sorry, but based on the context provided, there is no specific mention of "RAG architecture." If you have more context or details about what RAG architecture refers to, I can try to provide a more accurate answer. Otherwise, I don\'t have information on that specific term. Let me know if you have any other questions! 😊'

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "what are the Deep Learning techinques used in this?"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'In the context provided, the Deep Learning techniques used in the project for Driver Facial Behaviour Analysis include:\n\n1. Using deep learning techniques to extract facial features and classify images into behavior categories.\n2. Splitting the dataset into training, validation, and testing sets.\n3. Optimizing hyperparameters to achieve high accuracy and reduce misclassifications.\n\nThese techniques involve leveraging deep neural networks to analyze and classify facial behavior data for driver monitoring and safety enhancement. If you have any more questions or need further clarification, feel free to ask! 😊'

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "can you list down"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'Certainly! Here is a list of Deep Learning techniques used in the Driver Facial Behaviour Analysis project:\n\n1. Convolutional Neural Networks (CNNs): Utilized for image-based data processing and feature extraction from facial images.\n2. Recurrent Neural Networks (RNNs): Potentially used for analyzing sequential data, such as tracking eye movements over time.\n3. Transfer Learning: Leveraged to fine-tune pre-trained models for facial behavior classification tasks.\n4. Hyperparameter Optimization: Adjusting parameters like learning rate, batch size, and network architecture to improve model performance.\n5. Data Augmentation: Increasing the diversity of the training data by applying transformations like rotation, scaling, and flipping to improve model generalization.\n\nThese techniques collectively contribute to the development of an effective deep learning model for identifying unsafe driver behaviors through facial analysis. If you have any more questions or need additional detail

In [ ]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
    persist_directory="vector_store"  # ✅ this automatically persists in Chroma 1.1.1
)

print("✅ Vector store saved!")


✅ Vector store saved!


In [ ]:
vectordb = Chroma(
    persist_directory="vector_store",
    embedding_function=embedding_model
)


C:\Users\msi\AppData\Local\Temp\ipykernel_18204\2617956255.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(
